In [39]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib.colors import TwoSlopeNorm


In [40]:
df = pd.read_csv('with LongLat_rainfall_trend_results.csv')

# Convert degrees and minutes to decimal degrees for latitude and longitude
df['latitude'] = df['Lat (Deg)'] + df['Lat (Min)'] / 60
df['longitude'] = df['Long (Deg)'] + df['Long (Min)'] / 60

# Load Bangladesh boundary
bd_gdf = gpd.read_file('geoBoundaries-BGD-ADM0_simplified.geojson')


In [42]:
#Prepare data for interpolation
x = df['longitude'].values
y = df['latitude'].values


IDW METHOD 

In [ ]:
# Loop through each month
for p in range(3, 5):  # For months 1 to 12
    month_name = str(p)
    z = df[month_name].values  # Replace with the correct column for data
    
    print(z)
    
    # Create a grid for interpolation
    krig_bounds_xmin = bd_gdf.bounds.minx.iloc[0]
    krig_bounds_xmax = bd_gdf.bounds.maxx.iloc[0]
    krig_bounds_ymin = bd_gdf.bounds.miny.iloc[0]
    krig_bounds_ymax = bd_gdf.bounds.maxy.iloc[0]

    # Grid resolution
    horizontal_distance = krig_bounds_xmax - krig_bounds_xmin
    horizontal_resolution = 300  # Higher = more time/memory
    use_resolution = horizontal_distance / horizontal_resolution

    gridx = np.arange(krig_bounds_xmin, krig_bounds_xmax, use_resolution)
    gridy = np.arange(krig_bounds_ymin, krig_bounds_ymax, use_resolution)
    xx, yy = np.meshgrid(gridx, gridy)

    # IDW Function
    def idw_interpolation(x, y, z, gridx, gridy, power=2):
        """Perform Inverse Distance Weighting (IDW) interpolation."""
        interpolated = np.zeros_like(xx, dtype=float)
        for i in range(xx.shape[0]):
            for j in range(xx.shape[1]):
                distances = np.sqrt((x - xx[i, j]) ** 2 + (y - yy[i, j]) ** 2)
                weights = 1 / (distances ** power + 1e-10)  # Avoid division by zero
                interpolated[i, j] = np.sum(weights * z) / np.sum(weights)
        return interpolated

    # Apply IDW
    z_interp = idw_interpolation(x, y, z, gridx, gridy)

    # Mask the data outside the boundary
    points = [Point(px, py) for px, py in zip(xx.flatten(), yy.flatten())]
    mask = np.array([bd_gdf.contains(point).any() for point in points]).reshape(xx.shape)
    z_interp_masked = np.ma.masked_array(z_interp, mask=~mask)

    # Ensure vmin, vcenter, and vmax are in ascending order
    vmin = z_interp_masked.min()
    vmax = z_interp_masked.max()

    # Adjust vmin and vmax if needed
    if vmin >= 0:
        vmin = -0.1  # Ensure vmin is less than 0 for proper divergence
    if vmax <= 0:
        vmax = 0.1  # Ensure vmax is greater than 0 for proper divergence

    # Create the TwoSlopeNorm
    norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

    # Plot the results
    fig, ax = plt.subplots(figsize=(10, 10))
    bd_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=1)

    im = ax.imshow(
        z_interp_masked,
        extent=[krig_bounds_xmin, krig_bounds_xmax, krig_bounds_ymin, krig_bounds_ymax],
        origin='lower',
        cmap='RdYlBu',  # Diverging colormap (Red-Yellow-Blue)
        #alpha=0.7,
        alpha=0.5,
        norm=norm,  # Apply TwoSlopeNorm
    )
    plt.colorbar(im, ax=ax, label='Interpolated Data')
    ax.scatter(x, y, c='red', s=30, edgecolor='black', label='Stations')

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'IDW Interpolation for Month {month_name}')
    ax.legend()
    plt.show()


Another method
